In [1]:
import scanpy as sc
import numpy as np
import pandas as pd

adata1 = sc.read_h5ad('/public/home/syj/scpDeconv/dataset/raw_data/human_palatine_tonsil_spatial_proteomics.h5ad')
adata1

/public/home/syj/miniconda3/envs/SpatialDec/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AnnData object with n_obs × n_vars = 2492 × 174
    obs: 'ID', 'X', 'Y', 'B', 'T', 'T4'
    var: 'name1', 'name2', 'name3', 'pid', 'ref', 'group'
    uns: 'celltype_gd'
    obsm: 'spatial'

In [2]:
adata1.uns['celltype_gd']

,B,T4
1,0.249629,0.194444
2,0.084695,0.164683
3,0.046062,0.125000
4,0.054978,0.037698
5,0.046062,0.069444
...,...,...
2488,0.271917,0.172619
2489,0.124814,0.067460
2490,0.102526,0.105159
2491,0.118871,0.134921


In [3]:
adata1.uns['celltype_gd']['Stromal cells'] = 1 - (adata1.uns['celltype_gd']['B'] + adata1.uns['celltype_gd']['T4'])
adata1.uns['celltype_gd']['Stromal cells'][adata1.uns['celltype_gd']['Stromal cells'] < 0] = 0.
adata1.uns['celltype_gd'].columns = ['B cells', 'T cells', 'Stromal cells']
adata1.uns['celltype_gd']

/tmp/ipykernel_3635770/3756537011.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  adata1.uns['celltype_gd']['Stromal cells'][adata1.uns['celltype_gd']['Stromal cells'] < 0] = 0.


,B cells,T cells,Stromal cells
1,0.249629,0.194444,0.555927
2,0.084695,0.164683,0.750622
3,0.046062,0.125000,0.828938
4,0.054978,0.037698,0.907324
5,0.046062,0.069444,0.884493
...,...,...,...
2488,0.271917,0.172619,0.555464
2489,0.124814,0.067460,0.807725
2490,0.102526,0.105159,0.792315
2491,0.118871,0.134921,0.746209


In [4]:
adata1.uns['celltype_gd'].sum(axis=1).sum()

2496.002741809948

In [5]:
def minmax_norm(row):
    return row / row.sum()

adata1.uns['celltype_gd'] = adata1.uns['celltype_gd'].apply(minmax_norm, axis=1)
adata1.uns['celltype_gd']

,B cells,T cells,Stromal cells
1,0.249629,0.194444,0.555927
2,0.084695,0.164683,0.750622
3,0.046062,0.125000,0.828938
4,0.054978,0.037698,0.907324
5,0.046062,0.069444,0.884493
...,...,...,...
2488,0.271917,0.172619,0.555464
2489,0.124814,0.067460,0.807725
2490,0.102526,0.105159,0.792315
2491,0.118871,0.134921,0.746209


In [6]:
adata1.uns['celltype_gd'].sum(axis=1).sum()

2492.0

In [7]:
adata2 = sc.read_h5ad('/public/home/syj/scpDeconv/dataset/raw_data/human_palatine_tonsil_reference_proteomics.h5ad')
adata2

AnnData object with n_obs × n_vars = 17367 × 174
    obs: 'celltype', 'barcode'
    var: 'name1', 'name2'

In [8]:
adata2.obs['celltype']

cell_1        Naive CD4 T
cell_2              CD4 T
cell_3              CD4 T
cell_4               GCBC
cell_5                NBC
                 ...     
cell_17363    Naive CD4 T
cell_17364    Naive CD4 T
cell_17365          CD8 T
cell_17366    Naive CD4 T
cell_17367    Naive CD4 T
Name: celltype, Length: 17367, dtype: category
Categories (19, object): ['Activated NBC', 'CD4 T', 'CD8 T', 'DC', ..., 'PC', 'PDC', 'cycling T', 'epithelial']

In [9]:
sc.pp.normalize_total(adata1, target_sum=1e4)
sc.pp.log1p(adata1)
sc.pp.normalize_total(adata2, target_sum=1e4)
sc.pp.log1p(adata2)

In [10]:
np.unique(adata2.obs['celltype'].values)

array(['Activated NBC', 'CD4 T', 'CD8 T', 'DC', 'DN', 'FDC', 'GCBC',
       'Granulocytes', 'ILC', 'MBC', 'Mono/Macro', 'NBC', 'NK',
       'Naive CD4 T', 'Naive CD8 T', 'PC', 'PDC', 'cycling T',
       'epithelial'], dtype=object)

In [11]:
class_mapping = {
    'Activated NBC': 'B cells', 'NBC': 'B cells', 'GCBC': 'B cells', 'MBC': 'B cells', 'PC': 'B cells',
    'CD4 T': 'T cells', 'Naive CD4 T': 'T cells', 'CD8 T': 'T cells', 'Naive CD8 T': 'T cells', 'cycling T': 'T cells', 'DN': 'T cells',
    'DC': 'Stromal cells', 'Granulocytes': 'Stromal cells', 'PDC': 'Stromal cells', 'epithelial': 'Stromal cells', 'FDC': 'Stromal cells',
    'ILC': 'Stromal cells', 'Mono/Macro': 'Stromal cells', 'NK': 'Stromal cells'
}

adata2.obs['celltype'] = adata2.obs['celltype'].map(class_mapping)

print(adata2.obs['celltype'])
print(np.unique(adata2.obs['celltype']))

cell_1        T cells
cell_2        T cells
cell_3        T cells
cell_4        B cells
cell_5        B cells
               ...   
cell_17363    T cells
cell_17364    T cells
cell_17365    T cells
cell_17366    T cells
cell_17367    T cells
Name: celltype, Length: 17367, dtype: object
['B cells' 'Stromal cells' 'T cells']


In [12]:
x1 = adata2.X.A
y1 = adata2.obs['celltype']
celltypes = np.unique(y1)

x, y = None, None
celltype_num = len(celltypes)
cells_sub = []

for i in range(celltype_num):
    cells_sub.append(x1[np.array(y1 == celltypes[i]), :])

for i in range(20000):

    # Create fractions for available celltypes
    fracs = np.random.rand(celltype_num)
    fracs_sum = np.sum(fracs)
    fracs = np.divide(fracs, fracs_sum)

    samp_fracs = np.multiply(fracs, 50)
    samp_fracs = np.array(list(map(round, samp_fracs)))

    artificial_samples = None

    for j in range(celltype_num):
        cells_fraction = np.random.randint(0, cells_sub[j].shape[0], samp_fracs[j])
        cells_choice = cells_sub[j][cells_fraction, :]
        if cells_choice.shape[0] > 0:
            artificial_samples = np.concatenate((artificial_samples, cells_choice)) if artificial_samples is not None else cells_choice
            

    cell = artificial_samples.mean(0).reshape(-1, 1)
    cell_y = (samp_fracs/samp_fracs.sum()).reshape(-1, 1)
    
    x = np.concatenate((x, cell), axis=1) if x is not None else cell
    y = np.concatenate((y, cell_y), axis=1) if y is not None else cell_y
    if i % 1000 == 0: print(i)

print(x.shape, y.shape)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
(174, 20000) (3, 20000)


In [13]:
df = pd.DataFrame(x.T, columns=adata2.var_names)
df

name2,B7H4,CD10,CD101,CD103,CD106,CD107a,CD112,CD117,CD11a,CD11b,...,LOX1,Podoplanin,TCR_Valpha24_Jalpha18,TCR_Valpha7_2,TCR_Vbeta13,TCR_Vdelta2,TCR_ab,TIGIT,TSLPR,XCR1
0,3.708046,3.458425,2.409513,2.004494,3.406426,4.179333,2.524414,2.568212,1.497214,0.0,...,3.156563,5.242797,4.757775,1.067066,4.425754,1.015527,3.744077,4.285648,3.052136,4.027991
1,4.021686,3.730115,2.293985,1.714585,3.624086,4.342383,2.396832,2.740033,0.905758,0.0,...,3.633704,5.168557,4.663737,1.050446,4.425462,0.477960,3.313441,4.359827,2.969178,3.703341
2,3.573237,3.821042,2.738730,2.281643,3.776618,4.254541,2.550593,3.120734,1.434546,0.0,...,3.022461,5.213113,4.519912,0.778451,4.319029,1.043990,3.116574,4.104668,3.000501,3.860883
3,3.832540,3.430689,2.759257,2.203160,3.715877,3.977196,2.708286,2.639401,1.262293,0.0,...,3.115266,5.135927,4.630448,0.869887,4.382441,1.325600,4.194292,4.547777,3.239063,4.009886
4,3.858782,3.395683,2.468228,2.051116,3.543293,4.185272,2.654134,2.959558,1.166371,0.0,...,3.229677,5.091561,4.638093,0.720044,4.263028,0.714690,3.272083,4.212565,3.175429,3.637689
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,3.917366,3.789325,2.703948,2.038498,3.634519,4.167222,2.576229,3.056338,1.219868,0.0,...,3.447010,5.196712,4.771171,0.815122,4.373089,0.889193,3.823706,4.407387,3.258865,4.115032
19996,3.481819,3.360815,2.850416,2.645734,3.355772,4.385104,2.560874,3.022099,1.380422,0.0,...,3.007959,5.204252,4.617874,1.052569,4.233635,1.071292,3.671311,4.378160,3.336398,3.970682
19997,3.846914,4.138356,2.880569,2.340056,3.478296,4.517397,2.815395,2.940415,1.303522,0.0,...,3.027623,5.328873,4.747675,0.695590,4.513898,0.728628,3.885917,4.381869,3.213026,3.923944
19998,3.480305,3.720337,2.482323,1.779567,3.792177,3.941871,2.837572,2.643756,1.148019,0.0,...,3.204226,5.093283,4.569626,0.676192,4.467453,0.508490,3.835464,4.559983,2.962552,3.824496


In [14]:
adata_0 = sc.AnnData(df)
for i in range(len(celltypes)):
    adata_0.obs[celltypes[i]] = y[i].T
adata_0.obs['batch'] = np.array(['0'] * adata_0.shape[0])
adata_0.uns['cell_types'] = celltypes

adata_0

/public/home/syj/miniconda3/envs/SpatialDec/lib/python3.9/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


AnnData object with n_obs × n_vars = 20000 × 174
    obs: 'B cells', 'Stromal cells', 'T cells', 'batch'
    uns: 'cell_types'

In [15]:
df2 = pd.DataFrame(adata1.X.A, columns=adata1.var_names)
df2

name2,B7H4,CD10,CD101,CD103,CD106,CD107a,CD112,CD117,CD11a,CD11b,...,LOX1,Podoplanin,TCR_Valpha24_Jalpha18,TCR_Valpha7_2,TCR_Vbeta13,TCR_Vdelta2,TCR_ab,TIGIT,TSLPR,XCR1
0,2.752201,2.752201,3.237181,0.000000,5.701896,5.420752,1.773658,1.773658,1.773658,0.000000,...,2.752201,4.372959,1.773658,0.000000,3.412935,0.000000,2.378132,3.562368,3.692347,3.023808
1,2.272848,2.272848,3.578731,0.000000,4.658560,5.708411,1.677751,2.913112,0.000000,0.000000,...,2.913112,3.889543,2.272848,0.000000,3.300309,0.000000,1.677751,3.889543,3.974849,3.125335
2,2.050922,3.335779,2.677600,0.000000,4.695021,5.700788,2.050922,2.050922,0.000000,2.050922,...,3.335779,3.335779,3.880026,0.000000,3.059890,0.000000,3.059890,3.729311,3.729311,3.729311
3,2.343362,2.343362,3.375831,0.000000,5.138686,5.742152,0.000000,0.000000,0.000000,0.000000,...,3.375831,3.375831,0.000000,0.000000,0.000000,0.000000,2.343362,4.451387,3.375831,3.375831
4,2.998714,3.273855,2.998714,0.000000,4.167168,5.464128,0.000000,0.000000,0.000000,0.000000,...,2.617869,3.489398,0.000000,0.000000,1.995141,0.000000,1.995141,4.261068,3.947891,2.998714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2487,2.960883,4.024370,1.718966,1.718966,5.714386,5.082996,2.318213,0.000000,0.000000,0.000000,...,3.173618,4.477295,1.718966,0.000000,3.348940,0.000000,0.000000,4.103037,4.175964,4.024370
2488,2.719385,1.744609,2.346347,0.000000,5.024915,5.195727,2.719385,2.346347,0.000000,0.000000,...,2.990452,4.508105,1.744609,0.000000,2.346347,0.000000,1.744609,3.773059,3.203491,3.658145
2489,2.466545,2.466545,2.466545,0.000000,4.585325,5.639505,1.854865,3.116327,0.000000,0.000000,...,3.116327,2.843310,1.854865,2.466545,3.506908,1.854865,0.000000,4.263818,3.506908,3.116327
2490,1.544570,2.966807,2.966807,1.544570,4.838838,5.584957,0.000000,2.966807,2.489717,0.000000,...,2.966807,3.417322,2.489717,0.000000,3.890134,0.000000,0.000000,3.962780,3.811793,3.417322


In [16]:
adata_1 = sc.AnnData(df2)
for i in range(len(celltypes)):
    adata_1.obs[celltypes[i]] = adata1.uns['celltype_gd'][celltypes[i]].values
adata_1.obs['batch'] = np.array(['1'] * adata_1.shape[0])
adata_1.uns['cell_types'] = celltypes

adata_1

/public/home/syj/miniconda3/envs/SpatialDec/lib/python3.9/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


AnnData object with n_obs × n_vars = 2492 × 174
    obs: 'B cells', 'Stromal cells', 'T cells', 'batch'
    uns: 'cell_types'

In [17]:
adata_0.var_names_make_unique()
adata_1.var_names_make_unique()
adata = sc.concat((adata_0, adata_1))
adata.uns['cell_types'] = celltypes
adata

/public/home/syj/miniconda3/envs/SpatialDec/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


AnnData object with n_obs × n_vars = 22492 × 174
    obs: 'B cells', 'Stromal cells', 'T cells', 'batch'
    uns: 'cell_types'

In [18]:
adata.write_h5ad('/public/home/syj/scpDeconv/dataset/mixup_data/human_palatine_tonsil.h5ad') # Please modify this to your address.